In [3]:
import sys
import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf
import platform

print(f"Python Platform: {platform.platform()}")
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Python Platform: macOS-13.2.1-arm64-arm-64bit
Tensor Flow Version: 2.12.0
Keras Version: 2.12.0

Python 3.10.10 | packaged by conda-forge | (main, Mar 24 2023, 20:12:31) [Clang 14.0.6 ]
Pandas 2.0.0
Scikit-Learn 1.2.2
GPU is available


In [4]:
# IMPORTING STATEMENTS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import mutual_information_regression as mri
import helper_functions_fs as hf

np.random.seed(7)

In [5]:
from sklearn.preprocessing import StandardScaler

# Import the Data

In [6]:
# training data
data = pd.read_csv('train.csv', index_col=0)
data.head()

,InterestRate,MinorityRatio,CensusTractMedFamIncome,MonthlyIncome,HUDMedIncome,LTV,PaymentCount,Age1,Age2,HousingExpense,...,State_31,State_39,State_42,State_48,State_55,State_infrequent_sklearn,Minority_Min,Minority_Missing,Minority_White,Borrower2Dummy
170846,3.750,5.631,96500,4693,69400,72.0,360,65,61,21.76,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1
154335,4.375,5.745,93258,3194,94200,79.0,360,34,0,30.74,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0
145818,2.750,21.300,82260,6344,81700,99.0,360,32,56,27.87,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
94035,3.375,4.000,71172,9776,76700,80.0,360,25,25,9.00,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
175348,4.250,8.000,65755,3800,73100,80.0,360,30,0,12.00,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0


In [7]:
data.shape

(189079, 74)

In [8]:
# defining response variable and input features
y = data.pop('InterestRate')
X = data.copy()

In [9]:
# test data
data_test = pd.read_csv('test.csv', index_col=0)
data_test.head()

,InterestRate,MinorityRatio,CensusTractMedFamIncome,MonthlyIncome,HUDMedIncome,LTV,PaymentCount,Age1,Age2,HousingExpense,...,State_31,State_39,State_42,State_48,State_55,State_infrequent_sklearn,Minority_Min,Minority_Missing,Minority_White,Borrower2Dummy
188315,4.000,47.950,80357,18334,95900,80.0,360,39,0,17.07,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0
104813,3.250,6.489,112882,10065,86200,72.0,360,64,66,20.14,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1
164647,3.475,9.661,118045,19272,82300,72.0,360,62,62,14.70,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1
110948,3.125,5.172,100096,5171,89200,80.0,180,43,0,25.31,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
152548,4.750,4.396,91638,14262,79600,80.0,360,39,31,12.66,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1


In [10]:
y_test = data_test.pop('InterestRate')
X_test = data_test.copy()

# Scaling

In [11]:
# input variables
ss = StandardScaler()
X_train = ss.fit_transform(X)
X_test = ss.transform(X_test)
X = pd.DataFrame(X_train, columns=data.columns)
X_test = pd.DataFrame(X_test, columns=data.columns)
X.describe()

,MinorityRatio,CensusTractMedFamIncome,MonthlyIncome,HUDMedIncome,LTV,PaymentCount,Age1,Age2,HousingExpense,DebtExpense,...,State_31,State_39,State_42,State_48,State_55,State_infrequent_sklearn,Minority_Min,Minority_Missing,Minority_White,Borrower2Dummy
count,1.890790e+05,1.890790e+05,1.890790e+05,1.890790e+05,1.890790e+05,1.890790e+05,1.890790e+05,1.890790e+05,1.890790e+05,1.890790e+05,...,1.890790e+05,1.890790e+05,1.890790e+05,1.890790e+05,1.890790e+05,1.890790e+05,1.890790e+05,1.890790e+05,1.890790e+05,1.890790e+05
mean,-2.335544e-16,4.329118e-17,6.898592e-17,-1.819958e-16,6.964825e-16,2.689164e-16,-2.288664e-16,4.235170e-17,1.323913e-16,6.650381e-16,...,4.911594e-17,6.713514e-17,4.160011e-17,-5.148343e-17,2.324270e-17,3.370849e-17,-4.154375e-17,2.621145e-17,1.026286e-16,3.848105e-17
std,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,...,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00
min,-9.829129e-01,-2.968603e+00,-1.541693e+00,-3.136269e+00,-8.617748e-01,-3.667909e+00,-2.003581e+00,-1.101219e+00,-2.410365e+00,-3.148844e+00,...,-2.210030e-01,-3.043770e-01,-2.002391e-01,-1.994807e-01,-3.706281e-01,-5.073212e-01,-2.293908e-01,-2.695680e-01,-2.737468e+00,-1.210046e+00
25%,-6.899229e-01,-6.907823e-01,-6.998868e-01,-6.839963e-01,-8.617748e-01,-2.778056e-01,-8.066413e-01,-1.101219e+00,-7.202951e-01,-7.560665e-01,...,-2.210030e-01,-3.043770e-01,-2.002391e-01,-1.994807e-01,-3.706281e-01,-5.073212e-01,-2.293908e-01,-2.695680e-01,3.653010e-01,-1.210046e+00
50%,-3.806037e-01,-2.074440e-01,-2.267679e-01,-6.296614e-02,-9.168695e-02,5.697203e-01,-1.333627e-01,1.920703e-01,-1.531912e-01,8.581993e-03,...,-2.210030e-01,-3.043770e-01,-2.002391e-01,-1.994807e-01,-3.706281e-01,-5.073212e-01,-2.293908e-01,-2.695680e-01,3.653010e-01,8.264146e-01
75%,2.836096e-01,5.193869e-01,4.021168e-01,5.102925e-01,1.650090e-01,5.697203e-01,6.895334e-01,7.761363e-01,5.862923e-01,7.836339e-01,...,-2.210030e-01,-3.043770e-01,-2.002391e-01,-1.994807e-01,-3.706281e-01,-5.073212e-01,-2.293908e-01,-2.695680e-01,3.653010e-01,8.264146e-01
max,3.195775e+00,4.143399e+00,3.701777e+00,3.949844e+00,5.170580e+00,5.697203e-01,3.981118e+00,2.862086e+00,4.372398e+00,5.361121e+00,...,4.524826e+00,3.285399e+00,4.994029e+00,5.013015e+00,2.698123e+00,1.971138e+00,4.359372e+00,3.709638e+00,3.653010e-01,8.264146e-01


In [12]:
# dependent variables
ss_y = StandardScaler()
y = ss_y.fit_transform(np.array(y).reshape(-1, 1))
y_test = ss_y.transform(np.array(y_test).reshape(-1, 1))
pd.Series(y.flatten()).describe()

count    1.890790e+05
mean    -1.981173e-16
std      1.000003e+00
min     -2.684678e+00
25%     -7.623710e-01
50%     -1.856790e-01
75%      5.832437e-01
max      4.427857e+00
dtype: float64

# TODO:  PREPARATION FOR VARIABLE SELECTION APPLICATIONS 

Below is a list of variables we are choosing to keep for the purposes of re-running our models on the selected variables:
- MinorityRatio
- HUDMedIncome 
- PaymentCount
- NoteDate
- FirstTime
- SelfEmployed
- LoanPurpose_1

(see notebook *2_feature_selection.ipynb*)

In [13]:
X_vs = X[['HUDMedIncome', 'PaymentCount', 'NoteDate', 'FirstTime', 'SelfEmployed', 'LoanPurpose_1']]
X_test_vs = X_test[['HUDMedIncome', 'PaymentCount', 'NoteDate', 'FirstTime', 'SelfEmployed', 'LoanPurpose_1']]

# TODO: Scoring Methods

We define a helper function to save the scores for each model and make a final comparison.

The metrics we will analyse are: 
- MSE: Mean Squared Error
- MAE: Mean Absolute Error
- MR: Mean Residual
- **R2 ???????**
- **FAIRNESS QUALCOSA!!! TODO!!**

# PRENDERE DA ONE NOTE !!!!

In [44]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error


def mean_residual(y_true, prediction):
    '''Compute the mean residual metric'''
    residuals = (y_true - prediction)
    MR = residuals.mean()
    return MR


def compute_scores(x, y_true, modelname):
    '''Compute the following metrics for the given model: MSE, MAE, MR, r2'''
    prediction = modelname.predict(x)
    y_true = y_true.reshape(prediction.shape)

    MSE = mean_squared_error(y_true, prediction)
    print(f"The mean squared error of the model is {MSE:.4f}")
    MAE = mean_absolute_error(y_true, prediction)
    print(f"The mean absolute error of the model is {MAE:.4f}")
    MR = mean_residual(y_true, prediction)
    print(f"The mean residual of the model is {MR:.4f}")
    R2 = r2_score(y_true, prediction)
    adj_R2 = 1- ((1-R2) * (X.shape[0]-1)/(X.shape[0]-X.shape[1]-1))
    print(f"The adjusted R-squared of the model is {adj_R2:.4f}")
    return [MSE, MAE, MR, adj_R2]


def save_scores(model, scores):
    '''Store scores for the given model into a dedicated dataframe for a final comparison across models'''
    MSE, MAE, MR, r2 = scores
    pd.Series(scores).to_csv('scores/'+model)
    model_names.append(model)
    MSEs.append(MSE)
    MAEs.append(MAE)
    MRs.append(MR)
    r2s.append(r2)
    

def display_scores():
    scores_df = pd.DataFrame([model_names, MSEs, MAEs, MRs, r2s], 
                 index=['method', 'MSE', 'MAE', 'MR', 'adj R-squared']).T
    return scores_df

In [45]:
model_names, MSEs, MAEs, MRs, r2s = [],[],[],[],[]

In [46]:
y.flatten()

array([ 0.58324369,  1.54439705, -0.95460169, ..., -1.33906304,
        0.00655167, -1.53129371])

# MODELS

# 1. Linear Regression

In [47]:
import statsmodels.api as sm

In [48]:
#add constant to predictor variables
x = sm.add_constant(X)
#fit linear regression model
lr = sm.OLS(y, x).fit()

In [49]:
#view model summary
print(lr.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.705
Model:                            OLS   Adj. R-squared:                  0.705
Method:                 Least Squares   F-statistic:                     7660.
Date:                Wed, 26 Jul 2023   Prob (F-statistic):               0.00
Time:                        17:45:45   Log-Likelihood:            -1.5285e+05
No. Observations:              189079   AIC:                         3.058e+05
Df Residuals:                  189019   BIC:                         3.064e+05
Df Model:                          59                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const     

In [50]:
X_TEST = sm.add_constant(X_test)
scores = compute_scores(X_TEST, y_test, lr)
save_scores('LinearRegression', scores)

The mean squared error of the model is 0.2967
The mean absolute error of the model is 0.4255
(47270,) (47270,)
The mean residual of the model is 0.0002
The adjusted R-squared of the model is 0.7078


### Linear Regression with selected features

In [51]:
lr_vs = linear_model.LinearRegression().fit(X_vs, y)

In [52]:
scores = compute_scores(X_test_vs, y_test, lr_vs)
save_scores('LinearRegression_VS', scores)

The mean squared error of the model is 0.3287
The mean absolute error of the model is 0.4457
(47270, 1) (47270, 1)
The mean residual of the model is 0.0005
The adjusted R-squared of the model is 0.6763


# 2. RANSAC

https://scikit-learn.org/stable/auto_examples/linear_model/plot_ransac.html#sphx-glr-auto-examples-linear-model-plot-ransac-py

In [53]:
from sklearn import linear_model

In [54]:
# Robustly fit linear model with RANSAC algorithm to avoid outlier interference
ransac = linear_model.RANSACRegressor().fit(X, y)

In [55]:
scores = compute_scores(X_test, y_test, ransac)
save_scores('RANSAC', scores)

The mean squared error of the model is 78524736458860731039744.0000
The mean absolute error of the model is 48409607505.2395
(47270, 1) (47270, 1)
The mean residual of the model is -46786291401.9595
The adjusted R-squared of the model is -77320855167889162895360.0000


### RANSAC with selected features

In [56]:
ransac_vs = linear_model.RANSACRegressor().fit(X_vs, y)

In [57]:
scores = compute_scores(X_test_vs, y_test, ransac_vs)
save_scores('RANSAC_VS', scores)

The mean squared error of the model is 0.3493
The mean absolute error of the model is 0.4567
(47270, 1) (47270, 1)
The mean residual of the model is 0.0323
The adjusted R-squared of the model is 0.6560


# 3. DNN

*Wang 2022:*

- three hidden layers are used with 10 nodes in the first hidden layer, 8 nodes in the second hidden layer, and 6 nodes in the third hidden layer. 
- All layers use tanh activation function. 
- Mean square error is used as the cost function. 
- Stochastic gradient descent algorithm with a batch size of 64 and epoch of 100 is used to train the weights. 

## Neural Networks
- forse non devo fare validation set? 
- sistemare con test

For neural networks, we experiment with different network configurations and pick the best in terms of 
# #TODO: scegliere metriche

on the validation set. 


In [80]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Activation

In [81]:
def create_model(layers, activation='tanh', dropout_rate=0.15, X_train=X):
    model = None
    model = Sequential()

    for i, nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes, input_dim = X_train.shape[1]))
            model.add(Activation(activation))
            model.add(Dropout(dropout_rate, seed=0))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
            model.add(Dropout(dropout_rate, seed=0))

    # final prediction
    model.add(Dense(1,activation=activation)) 
    
    # compile model
    model.compile(optimizer='sgd', # stochastic gradient descent
                  loss='mean_squared_error', # loss function chosen
                  metrics=['mean_squared_error']) # TODO !
    return model

In [84]:
layers = [[10,8,6] ,[121,53,77],[121,77,53],  [10,8,25,6] , [22,9, 19, 98]]#,  [120,53,92,40], [121,53,77, 91]]
activations = ['tanh']#['sigmoid', 'relu', 'tanh'] # don't need

In [ ]:
scores2 = pd.DataFrame(columns=['layers','activation','val_mean_squared_error'], index=np.arange(len(layers)*len(activations)))

for i, lay in enumerate(layers):
    print(f"Layer configuration number {i}: {lay}")
    for j, act in enumerate(activations):
        print(f"activation: {act} \n")
        model = create_model(lay, activation=act)
        
        history = model.fit(X, y,# validation_data=(X_val,y_val), 
                            epochs=50, batch_size=128, verbose=1
                           )
        scores2.iloc[i+j,:] = [lay, act,  history.history['mean_squared_error'][-1]]
        path = 'saved_nn/model' + str(i)+'.'+str(j)
        model.save(path)
        print('\n\n')

Layer configuration number 0: [10, 8, 6]
activation: tanh 

Epoch 1/50
1478/1478 [==============================] - 7s 4ms/step - loss: 0.5547 - mean_squared_error: 0.5547
Epoch 2/50
1478/1478 [==============================] - 6s 4ms/step - loss: 0.4145 - mean_squared_error: 0.4145
Epoch 3/50
1478/1478 [==============================] - 6s 4ms/step - loss: 0.3988 - mean_squared_error: 0.3988
Epoch 4/50
1478/1478 [==============================] - 6s 4ms/step - loss: 0.3918 - mean_squared_error: 0.3918
Epoch 5/50
1478/1478 [==============================] - 6s 4ms/step - loss: 0.3877 - mean_squared_error: 0.3877
Epoch 6/50
 470/1478 [========>.....................] - ETA: 4s - loss: 0.3860 - mean_squared_error: 0.3860

In [ ]:
scores2

# TODO: reimport the chosen model and predict


In [78]:
scores = compute_scores(X_test, y_test, model)
save_scores('NN', scores)

1478/1478 [==============================] - 2s 1ms/step
The mean squared error of the model is 0.3089
The mean absolute error of the model is 0.4237
The mean residual of the model is 0.0144
The adjusted R-squared of the model is 0.6958


In [79]:
display_scores()

,method,MSE,MAE,MR,adj R-squared
0,LinearRegression,0.296738,0.425545,0.000606,0.707812
1,LinearRegression_VS,0.32875,0.445652,0.000465,0.67629
2,RANSAC,0.565692,0.570139,-0.056636,0.442981
3,RANSAC_VS,0.423814,0.488471,0.154729,0.582684
4,NN,0.342096,0.449317,-0.000316,0.663148
5,NN,0.314712,0.428407,0.015757,0.690113
6,NN,0.308915,0.423737,0.014423,0.695821


# FINAL SUMMARY OF METHODS PERFORMANCES

We display below a summary of the scores obtained by the different models. 
We see that models can be classified differently based on the metric of evaluation chosen. 




# 4. Support Vector Regression

In [21]:
from sklearn.svm import SVR
svr = SVR(kernel = 'rbf', verbose=True)
svr.fit(X, y)

/Users/fra/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[LibSVM]................................................................................................................................................................................*..............................................................................*..*
optimization finished, #iter = 256322
obj = -46893.235482, rho = -0.222898
nSV = 152603, nBSV = 137551


SVR(verbose=True)

In [31]:
scores = compute_scores(X_test, y_test, svr)
save_scores('SVR', scores)

The mean squared error of the model is 0.2141
The mean absolute error of the model is 0.3547
The mean residual of the model is 0.0194
The adjusted R-squared of the model is 0.7892


- fare subsamples e cercare margine migliore su uno solo, parametri:

    - Cfloat, default=1.0: Regularization parameter. The strength of the regularization is inversely proportional to C. Must be strictly positive. The penalty is a squared l2 penalty.

    - epsilonfloat, default=0.1: Epsilon in the epsilon-SVR model. It specifies the epsilon-tube within which no penalty is associated in the training loss function with points predicted within a distance epsilon from the actual value. Must be non-negative

# KERNEL RIDGE REGRESSION 

In [13]:
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV

kr = GridSearchCV(
    KernelRidge(kernel="rbf", gamma=0.1),
    param_grid={"alpha": [1e0, 0.1, 1e-2, 1e-3], "gamma": np.logspace(-2, 2, 5)}, verbose=3)

In [ ]:
import time

t0 = time.time()
kr.fit(X, y)
kr_fit = time.time() - t0
print(f"Best KRR with params: {kr.best_params_} and R2 score: {kr.best_score_:.3f}")
print("KRR complexity and bandwidth selected and model fitted in %.3f s" % kr_fit)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


In [ ]:
t0 = time.time()
y_kr = kr.predict(X_test)
kr_predict = time.time() - t0
print("KRR prediction for %d inputs in %.3f s" % (X_plot.shape[0], kr_predict))

# analisi finali
- predictions per anno vs reale (grafica)